<a href="https://colab.research.google.com/github/BroccoliWarrior/transformer-basic-knowledge/blob/main/Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Mask***

用于在MHA的计算中屏蔽掉某些不可见或不相关的位置，保证模型的正确性和有效性

意义：

    * 限制信息泄露：mask掉未来时刻的词，使得模型只能通过历史信息进行预测
    * 避免无关位置的干扰：对于batch中不同长度的序列，利用Padding来对其长度，然后利用mask将这些不包含实际意义的位置掩盖，避免在注意力计算中干扰模型
    * 训练目标需要：例如在BERT中，需要mask或随机token替换序列中的一部分，达到“MLM”的任务需要
    * 自定义需求


通常作用在注意力分数计算之后：

  $\text{Attention}(Q, K, V) = \text{Softmax}\left( \frac{QK^T}{\sqrt{d_k}} + M \right) V$

## ***Mask的类型***

### **1.Padding Mask**

  将序列长度对其的多余位置进行屏蔽

  如果某个token在索引$j$处是Padding，那么就将$M_{ij}=-∞$

  在计算了$QK^T$之后，根据输入序列的实际长度是否为Padding值判断，填充mask矩阵

### **2.Casual Mask（Look-Ahead Mask）**
  
  在解码的阶段，解码器需要一次只生成当前token，不能访问还未生成的未来token，以确保预测是自回归、符合语言生成的因果顺序

  Look-Ahead Mask一般是一个**上三角矩阵**（或下三角）

  例如，对长度$L$的序列，下标$(i,j)$表示第$i$个token能否看到第$j$个token。若$j>i$，则表示未来的位置需要屏蔽$(M_{ij}=-∞)$;反之，保持为$0$

  $M_{ij} =
\begin{cases}
0, & j \leq i \\
-\infty, & j > i
\end{cases}$

### **3.MLM Mask**

通过随机掩盖输入序列中的部分 token，然后让模型去预测这些被掩盖的 token 是什么，以此来学习语言的语义表示。例如在BERT、RiBERTa等模型的预训练中就采用了这种机制，它能帮助模型更好地捕捉上下文信息，理解单词在不同语境下的含义。

    * 将一部分token用[MASK]或随机词替换
    * Mask矩阵：在注意力机制中，通过不会完全的屏蔽被[MASK]的位置，因为它需要从其它位置获取信息。但有时为了避免模型“看到自己”，需要一定的自注意力屏蔽策略
    * 损失计算：只对被[MASK]的位置计算预测损失

In [1]:
import torch

def create_padding_mask(seq, pad_token=0):
    """生成 Padding Mask（填充掩码）"""
    # 标记填充位置（True表示需要屏蔽）
    mask = (seq == pad_token).bool()  # 形状: [batch_size, seq_len]
    # 扩展维度以适配注意力计算: [batch_size, 1, 1, seq_len]
    # 广播broadcast后变为: [batch_size, 1, seq_len_q, seq_len_k]
    return mask.unsqueeze(1).unsqueeze(2)

def create_casual_mask(seq_len):
    """生成 Casual Mask（前瞻掩码）"""
    # 创建上三角矩阵（对角线以上为True，表示需要屏蔽未来信息）
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    # 扩展维度以适配批量处理: [1, 1, seq_len, seq_len]
    return mask.unsqueeze(0).unsqueeze(0)

def combine_masks(seq, pad_token=0):
    """合并 Padding Mask 和 Casual Mask"""
    batch_size, seq_len = seq.size()

    # 生成两种掩码
    padding_mask = create_padding_mask(seq, pad_token)  # [batch_size, 1, 1, seq_len]
    casual_mask = create_casual_mask(seq_len)           # [1, 1, seq_len, seq_len]

    # 逻辑或运算合并掩码（任一掩码为True则结果为True）
    combined_mask = padding_mask | casual_mask          # [batch_size, 1, seq_len, seq_len]

    return combined_mask

# 示例用法
if __name__ == "__main__":
    # 输入序列：[batch_size, seq_len]，0表示填充符
    seq = torch.tensor([
        [1, 2, 3, 0, 0],  # 第1句：有效长度3
        [4, 5, 0, 0, 0],  # 第2句：有效长度2
        [6, 0, 0, 0, 0]   # 第3句：有效长度1
    ])

    # 生成合并掩码
    mask = combine_masks(seq, pad_token=0)
    print("合并掩码形状:", mask.shape)  # [3, 1, 5, 5]

    # 打印第1个样本的掩码（简化展示）
    print("\n第1个样本的掩码矩阵:")
    print(mask[0, 0].numpy())  # [5, 5] 矩阵，True表示需要屏蔽的位置


合并掩码形状: torch.Size([3, 1, 5, 5])

第1个样本的掩码矩阵:
[[False  True  True  True  True]
 [False False  True  True  True]
 [False False False  True  True]
 [False False False  True  True]
 [False False False  True  True]]
